In [2]:
import elasticsearch
import unicodedata
import csv
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
import networkx as nx
import pandas as pd
import pickle

es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])
#res = Search(using=es, index="filebeat*").query("match", username="root")
res = Search(using=es, index="packetbeat*")\
        .query('bool', filter=Q('exists', field='ip') & Q('exists', field='port')\
               & Q('exists', field='client_ip') & Q('exists', field='client_port')\
               & Q('exists', field='type') & Q('exists', field='method') & Q('exists', field='status')\
               & Q('exists', field='bytes_in') & Q('exists', field='bytes_out')
              )


#    .query("match", ip="/[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}/")
response = res.execute()
#filtered=res.filter("exists", field='ip' & "exists", field='client_ip').count()
#print(filtered)

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

#h = response.hits[0]
#print('/%s/%s/%s returned with score %f' % (
#    h.meta.index, h.meta.doc_type, h.meta.id, h.meta.score))

#df = pd.DataFrame((d.to_dict() for d in res))

Filtered logs: 15626969
Total number of logs: 15626969


In [3]:
df_tcp_udp = pd.DataFrame(((hit["@timestamp"],hit['type'],hit['method'],hit['direction'],hit['status'],hit['ip'],hit['port'],hit['client_ip'],hit['client_port'],hit['bytes_in'],hit['bytes_out']) for hit in res.scan()),\
                    columns=['Timestamp','type','method','direction','status','ip','port','client_ip','client_port','bytes_in','bytes_out'])

In [4]:
df_tcp_udp.to_pickle("tcp_udp_dataframe.pkl")

In [5]:
es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])
#res = Search(using=es, index="filebeat*").query("match", username="root")
res = Search(using=es, index="packetbeat*")\
        .query('bool', filter=Q('exists', field='ip')\
               & Q('exists', field='client_ip')\
               & Q('exists', field='type') & Q('exists', field='status')\
               & Q('exists', field='bytes_in') & Q('exists', field='bytes_out')\
               & Q('match', type='ICMP')
              )

response = res.execute()

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

Filtered logs: 119442
Total number of logs: 119442


In [6]:
df_icmp = pd.DataFrame(((hit["@timestamp"],hit['type'],hit['status'],hit['ip'],hit['client_ip'],hit['bytes_in'],hit['bytes_out']) for hit in res.scan()),\
                    columns=['Timestamp','type','status','ip','client_ip','bytes_in','bytes_out'])


In [7]:
df_icmp.to_pickle("icmp_dataframe.pkl")

In [8]:
df_icmp[:-10]

,Timestamp,type,status,ip,client_ip,bytes_in,bytes_out
0,2018-11-29T00:03:36.095Z,icmp,OK,10.1.1.14,113.59.38.202,68,68
1,2018-11-29T00:03:36.598Z,icmp,OK,10.1.1.14,36.33.158.10,68,68
2,2019-01-03T02:30:19.172Z,icmp,OK,10.1.1.26,46.234.125.89,16,16
3,2018-11-29T00:03:34.059Z,icmp,OK,10.1.1.14,36.66.223.243,68,68
4,2018-11-29T00:03:34.580Z,icmp,OK,10.1.1.14,42.56.86.10,68,68
5,2018-12-10T00:00:00.771Z,icmp,OK,10.1.1.40,206.117.25.90,4,4
6,2018-12-16T00:00:08.358Z,icmp,OK,10.1.1.23,195.251.255.69,4,4
7,2018-11-29T00:03:31.467Z,icmp,OK,10.1.1.14,221.229.203.10,68,68
8,2018-11-29T00:03:31.473Z,icmp,OK,10.1.1.14,101.75.226.49,68,68
9,2018-11-29T00:03:31.560Z,icmp,OK,10.1.1.14,223.111.17.10,68,68


In [9]:
df_tcp_udp[:-10]

,Timestamp,type,method,direction,status,ip,port,client_ip,client_port,bytes_in,bytes_out
0,2018-11-08T13:16:35.488Z,http,GET,out,OK,169.254.169.254,80,10.1.1.42,49482,110,1958
1,2018-11-12T00:00:07.235Z,http,GET,out,OK,169.254.169.254,80,10.1.1.5,57360,110,1958
2,2018-11-16T16:26:28.878Z,http,GET,out,OK,169.254.169.254,80,10.1.1.24,54650,110,1958
3,2018-11-17T01:28:54.264Z,dns,QUERY,out,Error,10.1.1.2,53,10.1.1.18,45047,46,121
4,2018-11-19T00:00:35.015Z,http,GET,out,OK,169.254.169.254,80,10.1.1.30,46390,110,1958
5,2018-11-29T00:00:06.277Z,dns,QUERY,out,OK,10.1.1.3,53,10.1.1.4,45930,46,62
6,2018-12-06T00:00:05.038Z,http,GET,out,OK,169.254.169.254,80,10.1.1.7,57580,110,1958
7,2018-12-13T00:00:05.450Z,http,GET,out,OK,169.254.169.254,80,10.1.1.23,52082,110,1958
8,2018-12-15T00:00:09.267Z,dns,QUERY,out,OK,10.1.1.3,53,10.1.1.56,36967,46,62
9,2018-12-19T02:39:14.409Z,dns,QUERY,out,OK,10.1.1.2,53,10.1.1.6,59935,46,62
